In [185]:
import pandas as pd
import numpy as np

acceptable_range = 0.5
current_temp = 25
target_temp = 22
key_to_find = f"Dictionary for target temp: {target_temp}"
temperature = np.arange(0,1.1,0.1)


with open('saved_data/stored_dictionary.json', 'r') as f:
    stored_dictionary = json.load(f) 

print(type(stored_dictionary))
aircon_status_result = pd.read_csv("saved_data/aircon_status_W512.csv")


fan_status_columns = [col for col in aircon_status_result.columns if 'Fan_Status' in col]
unique_statuses = set()
for col in fan_status_columns:
    unique_statuses.update(aircon_status_result[col].dropna().unique())
fan_status_filitered = {status for status in unique_statuses if status != 'OFF'}
   
aircon_units = len([
    col for col in aircon_status_result.columns
    if "FC_Unit_" in col and "_Status" in col and "Fan" not in col
])

while key_to_find not in stored_dictionary:
    if (target_temp - current_temp) < 0:
        target_temp += 0.5
    else:
        target_temp -= 0.5
        
    key_to_find = f"Dictionary for target temp: {target_temp}"


def expandPath(row_index):
    unit_data = []
    for i in range(1, aircon_units + 1):
        unit_info = {
            "unit": i,
            "status": aircon_status_result[f"FC_Unit_{i}_Status"].iloc[row_index],
            "fan_status": aircon_status_result[f"FC_Unit_{i}_Fan_Status"].iloc[row_index],
            "set_point": aircon_status_result[f"FC_Unit_{i}_Set_Point"].iloc[row_index],
            "operation_mode": aircon_status_result[f"FC_Unit_{i}_Operation_Mode"].iloc[row_index],
        }
        unit_data.append(unit_info)
    return unit_data


def expandpath_withrandomness(row_index):
    unit_data = []
    randomness = np.round(np.random.choice(temperature, 1)[0], 1)  
    scaled_change = randomness * 0.5

    print(f"randomness: {randomness}, scaled_change: {scaled_change}")

    if randomness <= 0.3:
        unit_data = expandPath(row_index)
        print()

    elif 0.3 < randomness <= 0.6:
        unit_to_change = np.random.randint(1, aircon_units + 1)
        print(f"Unit to change: {unit_to_change}")
        
        current_set_point = aircon_status_result[f"FC_Unit_{unit_to_change}_Set_Point"].iloc[row_index]
        new_set_point = current_set_point + np.random.choice([-1, 1]) * scaled_change
        print(f"{current_set_point} changed to {new_set_point}")
        
        unit_data = expandPath(row_index) 
        unit_data[unit_to_change - 1]["set_point"] = new_set_point
        print()
    else:
        unit_to_change = np.random.randint(1, aircon_units + 1)
        print(f"Unit to change: {unit_to_change}")

        current_fan_status = aircon_status_result[f"FC_Unit_{unit_to_change}_Fan_Status"].iloc[row_index]
        current_set_point = aircon_status_result[f"FC_Unit_{unit_to_change}_Set_Point"].iloc[row_index]
        new_set_point = current_set_point + np.random.choice([-1, 1]) * scaled_change
        print(f"{current_set_point} changed to {new_set_point}")

                   
        fan_mode_to_change = np.random.choice(list(fan_status_filitered))

        while current_fan_status == fan_mode_to_change:
            fan_mode_to_change = np.random.choice(list(fan_status_filitered))
            print("was same")

        print(f"{current_fan_status} changed to {fan_mode_to_change}")
        
        unit_data = expandPath(row_index)  # Get the base unit data
        unit_data[unit_to_change - 1]["set_point"] = new_set_point  # Update the set point
        unit_data[unit_to_change - 1]["fan_status"] = fan_mode_to_change  # Update the fan mode
        print()

    
    return unit_data
        
def findClosestTemperature(current_temp, paths):
    """Find the index of the closest available temperature in the paths dictionary."""
    closest_temp_index = None
    smallest_difference = float('inf')
    
    for key, value in paths.items():
        if isinstance(value, dict) and 'starting_temp' in value:
            if value['starting_temp'] and value['path']:  #ensure that paths[index] has both values in 'starting_temp' and 'path'
                difference = abs(value['starting_temp'] - current_temp)
                if difference < smallest_difference:
                    smallest_difference = difference
                    closest_temp_index = key   
    return closest_temp_index



# Check if either key exists in stored_dictionary
if key_to_find in stored_dictionary:
    print(f"Using {key_to_find}")
    stored_dict_key = stored_dictionary[key_to_find]

    filtered_paths = {
        key: value for key, value in stored_dict_key.items()
        if isinstance(value, dict) and 'starting_temp' in value
        and abs(value['starting_temp'] - current_temp) < acceptable_range
        and value["path"]
    }
    aircon_settings_result = []


    if filtered_paths:
        # Find the path with the smallest factor
        smallest_factor_path = min(filtered_paths.keys(), key=lambda x: filtered_paths[x]['factor'])
        aircon_settings_result.append({'path': str(stored_dict_key[smallest_factor_path])})
        final_results = []


        for index, value in enumerate(stored_dict_key[smallest_factor_path]['path']):
            result = expandpath_withrandomness(value)
            timetaken_Seconds = int(stored_dict_key[smallest_factor_path]['time_taken'][index])
            final_results.append({
                'time_taken_seconds': timetaken_Seconds,
                'aircon_settings': result
            })
        aircon_settings_result.append({'results': final_results})
    else:
        print("No paths found within the acceptable range of current temp.")
        print("Finding the closest temperature in the algorithm.")


        closest_temp_index = findClosestTemperature(current_temp, stored_dict_key)
        
        if closest_temp_index is not None:
            print("Closest temperature found at index:", closest_temp_index , "\n")

            aircon_settings_result.append({'path': str(stored_dict_key[closest_temp_index])})

            final_results =[]
            
            for index, value in enumerate(stored_dict_key[closest_temp_index]['path']):
                result = expandpath_withrandomness(value)
                timetaken_Seconds = int(stored_dict_key[closest_temp_index]['time_taken'][index])
                final_results.append({
                    'time_taken_seconds': timetaken_Seconds,
                    'aircon_settings': result
                })
            aircon_settings_result.append({'results': final_results})

        else:
            print("No valid paths available, even for the closest temperature.")

else:
    print(f"{key_to_find} does not exist")


print(aircon_settings_result)


<class 'dict'>
Using Dictionary for target temp: 22.5
randomness: 0.4, scaled_change: 0.2
Unit to change: 4
23.4 changed to 23.599999999999998

randomness: 0.9, scaled_change: 0.45
Unit to change: 8
23.2 changed to 23.65
MID changed to HIGH

randomness: 0.6, scaled_change: 0.3
Unit to change: 1
23.4 changed to 23.7

randomness: 0.8, scaled_change: 0.4
Unit to change: 6
22.7 changed to 23.099999999999998
VERY HIGH changed to VERY LOW

[{'path': "{'energy_consumption': [0.3700000000003456, 0.5099999999993088, 0.3400000000001455, 0.34000000000060027], 'time_taken': ['219', '225', '169', '273'], 'starting_temp': 24.543, 'starting_humi': 54.812, 'ending_temp': 22.957, 'ending_humi': 59.312, 'path': [1193, 1242, 1816, 2327], 'factor': 0.1257351105591566}"}, {'results': [{'time_taken_seconds': 219, 'aircon_settings': [{'unit': 1, 'status': 'ON', 'fan_status': 'MID', 'set_point': 21.6, 'operation_mode': 'FAN'}, {'unit': 2, 'status': 'ON', 'fan_status': 'HIGH', 'set_point': 22.6, 'operation_mod

In [164]:
randomness = np.round(np.random.choice(temperature, 1)[0], 1)  
print(randomness)

0.0
